In [1]:
import pandas as pd

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .appName('spark-sql-practice') \
    .master('local') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/13 03:01:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/13 03:01:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
sales = spark.read.options(header='true', inferschema='true').csv('./data/FactInternetSales.csv')

In [5]:
sales.createOrReplaceTempView('Sales')

23/07/13 03:01:52 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
customer = spark.read.options(header='true', inferschema='true').csv('./data/DimCustomer.csv')

In [7]:
customer.createOrReplaceTempView('DimCustomer')

In [8]:
geog = spark.read.options(header='true', inferschema='true').csv('./data/DimGeography.csv')

In [9]:
geog.createOrReplaceTempView('DimGeography')

In [10]:
spdf_salesinfo = spark.sql('select * from Sales').dropna()

In [11]:
getcustgeo = '''
select CustomerKey, CountryRegionCode, EnglishCountryRegionName as CountryName,
StateProvinceCode, StateProvinceName, City, PostalCode, upper(City)as UpperCity
from DimCustomer c
inner join DimGeography g
on (c.geographykey = g.geographykey)
where countryregioncode = 'US'
'''

In [12]:
spdf_custgeo_us = spark.sql(getcustgeo)

In [13]:
spdf_custgeo_us.show()

+-----------+-----------------+-------------+-----------------+-----------------+--------------+----------+--------------+
|CustomerKey|CountryRegionCode|  CountryName|StateProvinceCode|StateProvinceName|          City|PostalCode|     UpperCity|
+-----------+-----------------+-------------+-----------------+-----------------+--------------+----------+--------------+
|      11012|               US|United States|               WA|       Washington|     Bremerton|     98312|     BREMERTON|
|      11013|               US|United States|               OR|           Oregon|       Lebanon|     97355|       LEBANON|
|      11014|               US|United States|               WA|       Washington|       Redmond|     98052|       REDMOND|
|      11015|               US|United States|               CA|       California|       Burbank|     91502|       BURBANK|
|      11016|               US|United States|               CA|       California|Imperial Beach|     91932|IMPERIAL BEACH|
|      11021|   

In [14]:
spdf_salesinfo = spdf_salesinfo.withColumn('TaxRate', spdf_salesinfo.TaxAmt / spdf_salesinfo.SalesAmount)

In [19]:
lpdf_salesinfo = spark.sql('''select * from Sales limit 20''').toPandas()

In [23]:
lpdf_salesinfo.describe()

,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderLineNumber,RevisionNumber,OrderQuantity,UnitPrice,ExtendedAmount,UnitPriceDiscountPct,DiscountAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight
count,20.000000,2.000000e+01,2.000000e+01,2.000000e+01,20.000000,20.0,20.00000,20.000000,20.0,20.0,20.0,20.000000,20.000000,20.0,20.0,20.000000,20.000000,20.000000,20.000000,20.000000
mean,322.600000,2.010389e+07,2.011011e+07,2.011011e+07,19563.950000,1.0,51.55000,6.750000,1.0,1.0,1.0,3244.532820,3244.532820,0.0,0.0,1929.991460,1929.991460,3244.532820,259.562630,81.113370
std,15.719247,4.171124e+03,1.472556e+00,1.472556e+00,6535.510478,0.0,45.34717,2.971354,0.0,0.0,0.0,874.194613,874.194613,0.0,0.0,531.179408,531.179408,874.194613,69.935556,21.854867
min,310.000000,2.010123e+07,2.011011e+07,2.011010e+07,11003.000000,1.0,6.00000,1.000000,1.0,1.0,1.0,699.098200,699.098200,0.0,0.0,413.146300,413.146300,699.098200,55.927900,17.477500
25%,311.000000,2.010123e+07,2.011011e+07,2.011011e+07,14273.500000,1.0,6.00000,4.000000,1.0,1.0,1.0,3399.990000,3399.990000,0.0,0.0,1912.154400,1912.154400,3399.990000,271.999200,84.999800
50%,313.500000,2.010123e+07,2.011011e+07,2.011011e+07,16576.500000,1.0,34.00000,8.500000,1.0,1.0,1.0,3578.270000,3578.270000,0.0,0.0,2171.294200,2171.294200,3578.270000,286.261600,89.456800
75%,338.000000,2.011010e+07,2.011011e+07,2.011011e+07,27602.250000,1.0,100.00000,9.000000,1.0,1.0,1.0,3578.270000,3578.270000,0.0,0.0,2171.294200,2171.294200,3578.270000,286.261600,89.456800
max,351.000000,2.011010e+07,2.011011e+07,2.011011e+07,28389.000000,1.0,100.00000,10.000000,1.0,1.0,1.0,3578.270000,3578.270000,0.0,0.0,2171.294200,2171.294200,3578.270000,286.261600,89.456800


In [25]:
q = '''select salesTerritoryKey, 
count(*) TotalSales,
avg(SalesAmount) AverageSales
from Sales
group by SalesTerritoryKey
'''

In [26]:
salesSummary = spark.sql(q)

In [27]:
salesSummary.show()

+-----------------+----------+------------------+
|salesTerritoryKey|TotalSales|      AverageSales|
+-----------------+----------+------------------+
|                1|      8993| 405.8563939954403|
|                6|      7620|259.55969318897314|
|                3|        20|150.04147999999984|
|                5|        39|313.81665641025626|
|                9|     13345| 678.9809355115657|
|                4|     12265| 466.2169435142324|
|                8|      5625| 514.5444156800735|
|                7|      5558|475.71387446929265|
|               10|      6906|491.12542874322526|
|                2|        27|241.94326666666652|
+-----------------+----------+------------------+

